In [16]:
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tslearn.utils import to_time_series_dataset
import biosppy.signals.ecg as ecg
import matplotlib.pyplot as plt
import neurokit2 as nk
import heartpy as hp
from statistics import mean
import functions
from multiprocessing import Pool
from datetime import time
from pandarallel import pandarallel
import functions
import scipy
import math
import pywt
import operator
from functions import *
import itertools
from sklearn.ensemble import ExtraTreesClassifier


In [17]:
folder = ""
raw = folder + "raw_data/"
x_train = pd.read_csv('x_train_preprocess.csv')
x_test = pd.read_csv('x_test_preprocess.csv')
y_train = pd.read_csv(raw + 'y_train.csv', index_col=['id'])
x_train = x_train.fillna(0)
x_test = x_test.fillna(0)

In [18]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5117 entries, 0 to 5116
Columns: 555 entries, Unnamed: 0 to nk_feat_13
dtypes: float64(554), int64(1)
memory usage: 21.7 MB


In [19]:
#save feature name
feature_name = x_train.columns

In [20]:
selector = SelectKBest(score_func=f_classif, k=150)
selector.fit(np.array(x_train), np.array(y_train).squeeze())
cols = selector.get_support(indices=True)

In [21]:
#scale data
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [29]:
#random forest
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, random_state=0)

In [30]:
#cross validation score
scores = cross_val_score(clf, x_train, np.ravel(y_train), cv=10, scoring='f1_micro',n_jobs=-1)
print(scores.mean())
print(scores.std())
#0.6880920529598826 / 0.01306019561569018 MinMaxScaler


0.6880920529598826
0.01306019561569018


In [31]:
clf.fit(x_train, np.ravel(y_train))

RandomForestClassifier(random_state=0)

In [32]:
#feature importance
importances = clf.feature_importances_
indices = np.argsort(importances)[::-1]
# Print the feature ranking
print("Feature ranking:")
for f in range(x_train.shape[1]):
    print("%d. feature %s (%f)" % (f + 1, feature_name[indices[f]], importances[indices[f]]))
    

Feature ranking:
1. feature nk_feat_11 (0.011125)
2. feature nk_feat_6 (0.006920)
3. feature nk_feat_12 (0.006649)
4. feature nk_feat_10 (0.004851)
5. feature median_20 (0.004504)
6. feature mu_48 (0.004428)
7. feature mu_50 (0.004388)
8. feature mu_46 (0.004286)
9. feature mu_51 (0.003934)
10. feature nk_feat_5 (0.003850)
11. feature median_19 (0.003819)
12. feature median_22 (0.003610)
13. feature mu_134 (0.003554)
14. feature mu_49 (0.003502)
15. feature nk_feat_7 (0.003482)
16. feature median_133 (0.003426)
17. feature nk_feat_4 (0.003341)
18. feature median_50 (0.003341)
19. feature mu_76 (0.003291)
20. feature median_21 (0.003261)
21. feature median_18 (0.003255)
22. feature mu_133 (0.003232)
23. feature mu_75 (0.003223)
24. feature mu_78 (0.003199)
25. feature mu_47 (0.003192)
26. feature mu_52 (0.003187)
27. feature median_66 (0.003163)
28. feature nk_feat_13 (0.003161)
29. feature mu_77 (0.003156)
30. feature mu_81 (0.003121)
31. feature median_24 (0.003037)
32. feature median

In [33]:
import pickle
filename = 'models/random_forest_model.sav'
pickle.dump(clf, open(filename, 'wb'))
